# Introduction #

In the ever-evolving landscape of financial markets, data-driven decision-making is becoming increasingly essential [source]. The integration of artificial intelligence (AI) into finance, particularly through the use of machine learning models, has shown significant potential in optimizing investment strategies [source]. However, the complexity of financial markets requires more than just individual models. An ensemble approach— leveraging multiple models to assess financial, economic, and sentiment data —may provide a more comprehensive and accurate view of market opportunities.

This project aims to explore the design and development of a hybrid ensemble AI system for investment analysis. By combining technical, fundamental, economic, and sentiment-based models, the system seeks to generate reliable buy and sell signals only when a consensus is reached. The goal is to enhance accuracy and profitability in trading decisions by reducing noise and improving the reliability of predictions.

In addition to building a practical financial tool, this project serves as a research endeavor to deepen understanding of AI ensemble methods, their application in finance, and their potential to outperform individual models or traditional strategies.

## Objectives ##

To design and implement a hybrid ensemble AI system that integrates multiple financial analysis models—technical, fundamental, economic, and sentiment-based—to provide data-driven investment signals. By requiring a threshold of consensus among models through the use of weighted voting, the system aims to improve decision-making accuracy and profitability in trading strategies.

## Research & Learning Objectives ##

To deepen the understanding of ensemble AI methods in financial markets, exploring the interaction between different model types and refining techniques for model integration, consensus mechanisms, and trading strategy optimization. This project also serves as a learning platform to enhance practical skills in AI development, evaluation, and deployment within the context of algorithmic trading.

# Methodology / Approach #

The primary focus of this project is on the AI-driven decision-making process rather than the mechanics of trading bot development. Therefore, we will leverage established trading bot libraries that allow for the easy definition and implementation of various trading strategies. Using these libraries provides several advantages:
- Ease of use: They are user-friendly and provide abstractions that allow us to focus on the AI aspects.
- Time-efficient: Utilizing pre-built libraries reduces the time needed to develop a functioning backtesting system.
- Project focus: Since the goal of this project is to experiment with and improve AI models, rather than creating a custom trading bot, these libraries allow us to dedicate more resources to the AI components.

Each model within the ensemble will be designed using modern AI techniques (e.g., deep learning, reinforcement learning, or hybrid models) and will be trained to function independently. The performance of each model will be benchmarked against established financial indices such as the S&P 500 or NASDAQ 100, ensuring that each model performs at a level competitive with general market trends.

Each model will operate independently, utilizing distinct data sets related to the target security, which will encompass technical, fundamental, and sentiment-based features. Each model will provide a prediction in the form of a buy signal. It is important to note that the system is not tasked with identifying sell signals; the goal is to identify optimal buy opportunities, with the assumption that sell strategies may be handled by other components or human decision-makers.

Once each model provides its buy signal, the outputs will be aggregated using a weighted voting system. This system allows for models with higher accuracy or reliability to have a greater influence on the final decision. A predefined threshold will be applied, and if the aggregate signal exceeds the threshold, the system will generate a buy recommendation. The threshold is adjustable and can be fine-tuned during testing to balance risk and reward.


## Technologies ##

lorem ipsum

## Expected Outcomes ##

lorem ipsum




In [1]:
# Libraries


# ===================================
# Data Processing Imports
# ===================================
import pandas as pd
import matplotlib.pyplot as plt
#%matplotlib inline
import random

# ===================================
# Trading Bot Imports
# ===================================
import backtrader as bt
import yfinance as yf
from datetime import datetime


# ===================================
# AI Architecture Imports
# ===================================

## Trading Bots ##

The first step in this research is the implementation of several trading bots. Each bot is designed to operate based on well established but different trading strategies. 

Each bot will independently generate trading data, including buy/sell signals, performance metrics, and market conditions, and fundamentals of the particular security. This data will serve as the training dataset for its corresponding AI model within the ensemble. This approach allows for a diverse range of inputs and methodologies, enhancing the overall robustness and adaptability of the system.



### S&P 500 tickers for trading bots ###

In [2]:
#sp500_tickers = ['AAPL', 'TSLA']

# Scrape the S&P 500 data from Wikipedia
url = "https://en.wikipedia.org/wiki/List_of_S%26P_500_companies"
table = pd.read_html(url)  # Extract all tables from the URL

# The first table contains the S&P 500 data
sp500_table = table[0]

# Extract the 'Symbol' column as a list of ticker symbols
sp500_tickers = sp500_table['Symbol'].tolist()

# Sample a random number of tickers from the S&P 500
#random_tickers = random.sample(sp500_tickers, 50)

#sp500_tickers = random_tickers

### Trading Strategy: Momentum based investing

In this strategy the bot trades based off current momentum trends in the market. It buys when a stock has positive momentum over some perioud of time and then sells when that momentum turns negative. This strategy has performed well over recent years [SOURCE] and has some key advantages and disadvantages:

Advantages
- Trend Following: Momentum strategies capitalize on established trends, potentially leading to higher returns during bull markets.

- Behavioral Finance: They exploit behavioral biases, such as the tendency for investors to overreact to news or trends, allowing for profits from mispriced assets.

- Simplicity: Momentum strategies can be relatively straightforward to implement, using clear signals based on price movement.

- Historical Success: Many studies suggest that momentum strategies have historically outperformed the market over certain time frames.

- Diversification: Momentum can be applied across various asset classes (stocks, commodities, currencies), offering diversification benefits.

Disadvantages

- Reversals: Momentum can lead to significant losses if a trend reverses unexpectedly, as past performance does not guarantee future results.

- Market Volatility: During volatile or sideways markets, momentum strategies may struggle, leading to whipsaw losses.

- High Turnover: These strategies often require frequent trading, which can lead to higher transaction costs and tax implications.

- Risk of Herding: Momentum strategies can contribute to market bubbles as many traders chase the same trends, leading to instability when trends reverse.

- Lagging Indicators: Momentum indicators may lag in signaling the reversal of trends, causing late entry or exit points that reduce potential profits.



In [3]:
class MomentumStrategy(bt.Strategy):
    params = (
        ('buy_momentum_period', 10),
        ('sell_momentum_period', 5)     
    )

    def __init__(self):
        #dictionary of momentum indicators of each stock
        self.buy_momentum = {}
        self.sell_momentum = {}
        for i, data in enumerate(self.datas):
            self.buy_momentum[data] = bt.indicators.Momentum(data.close, period=self.params.buy_momentum_period)
            self.sell_momentum[data] = bt.indicators.Momentum(data.close, period=self.params.sell_momentum_period)
    
    def next(self):
        # Loop through each stock and make trading decisions:
        for i, data in enumerate(self.datas):
            #buy condition
            if self.buy_momentum[data][0] > 0 and not self.getposition(data):
                #print(f"buying {data._name}")
                self.buy()
                
            #sell condition
            elif self.sell_momentum[data][0] < 0 and self.getposition(data):
                print(f"selling {data._name}")
                self.sell()

    

### Cerebro Engine ###
The cerebro engine will be used to produce backtesting results with the individual trading bots.

In [9]:

# Initialize Backtrader's cerebro
cerebro = bt.Cerebro()

# Create an empty DataFrame to store the combined data
combined_df = pd.DataFrame()

# Loop through each ticker and download its data
for ticker in sp500_tickers:
    try:
        # Download data from Yahoo Finance
        df = yf.download(ticker, start='2023-01-01', end='2024-01-01')

        # Check if the DataFrame is empty or missing data
        if df.empty:
            print(f"Skipping {ticker}: No data available")
            continue
        
        # Add a column to identify the ticker symbol
        df['Ticker'] = ticker
        
        # Append the data to the combined DataFrame
        combined_df = pd.concat([combined_df, df])
        
        # Prepare the data for Backtrader (make sure the index is in datetime format)
        df.index = pd.to_datetime(df.index)
        
        # Create a PandasData feed for Backtrader
        data = bt.feeds.PandasData(dataname=df)
        
        # Add the data to cerebro
        cerebro.adddata(data, name=ticker)
        print(f"Downloaded and added data for {ticker}")

    except Exception as e:
        print(f"ERROR for {ticker}: {e}")

# Save the combined DataFrame to a single CSV file
combined_df.to_csv("combined_sp500_data.csv")
print("Saved combined data to combined_sp500_data.csv")

# Add your strategy here if needed
# cerebro.addstrategy(MyStrategy)

# Run the backtest (optional)
# cerebro.run()


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


Downloaded and added data for MMM
Downloaded and added data for AOS


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


Downloaded and added data for ABT
Downloaded and added data for ABBV
Downloaded and added data for ACN


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


...

Downloaded and added data for WYNN
Downloaded and added data for XEL


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


Downloaded and added data for XYL
Downloaded and added data for YUM
Downloaded and added data for ZBRA


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


Downloaded and added data for ZBH
Downloaded and added data for ZTS
Saved combined data to combined_sp500_data.csv


In [5]:
#for data in cerebro.datas:
 #   print(data)

In [6]:
# Add the strategy
cerebro.addstrategy(MomentumStrategy)

# Set initial capital
cerebro.broker.setcash(100000)

    
# Run the backtest
cerebro.run()

# Plot results
#fig = cerebro.plot()
#plt.show(fig)

selling MMM
selling MMM
selling MMM
selling MMM
selling MMM
selling MMM
selling MMM
selling MMM
selling MMM
selling MMM
selling MMM
selling MMM
selling MMM


In [7]:
final_value = cerebro.broker.getvalue()
print(f"Final Portfolio Value: ${final_value:.2f}")

Final Portfolio Value: $123594.78
